# 데이터 불러오기 및 Train 데이터 살펴보기

In [3]:
# 필요 라이브러리 가져오기
import numpy as np
import pandas as pd
import dask.dataframe as dd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
train_dask_df = dd.read_csv('../input/amex-default-prediction/train_data.csv',blocksize="10MB")

In [4]:
print('Number of rows:', train_dask_df.shape[0].compute())
# 데이터의 대략적인 정보 확인
train_dask_df.info()

## observation
- 데이터 전체 샘플의 개수는 55314151개다. (샘플수는 고객의 수를 나타낸다)
- 총 190개의 컬럼을 가지고 있다. (S,D,P,B,R 총 다섯개의 카테고리의 컬럼들)
- 데이터 타입은 185개의 컬럼이 실수(대부분의 비식별 데이터), 4개가 문자열(customer ID, 날짜 등등), 1개의 정수로 이루어져 있다. 

In [5]:
train_df = pd.read_csv('../input/amex-default-prediction/train_data.csv', nrows=100000)
# train_df = train_df.drop('customer_ID', axis = 1)

In [6]:
print('Shape of dataset is:', train_df.shape)

# 10만개의 데이터 정보 확인
train_df.info()

### 데이터의 양이 너무 많기때문에 10만개의 데이터만 추출해서 EDA를 진행해 보기로 했다.

In [7]:
train_df

In [76]:
# 데이터 하나의 용량이 크기때문에 아래 함수를 사용해 용량을 줄여줌(float64 -> float32)
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 **2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == "object":
            pass
        elif dtype_name == "bool":
            df[col] = df[col].astype("int8")
        elif dtype_name.startswith("int") or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast="integer")
        else:
            df[col] = pd.to_numeric(df[col], downcast="float")
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f"{(100*(start_mem-end_mem)/start_mem): .1f}% 압축됨")
    return df

downcast(train_df)

In [80]:
display(train_df.info(verbose=True)) # float64가 모두 float32로 변환되었다.

## observation
- 전체적인 데이터의 구성은 비식별 데이터다(정확한 데이터의 의미를 알 수 없다)
- customer ID는 암호화 된 문자로 구성되어 있다.
- 몇몇 데이터에 결측치가 있는 것을 확인할 수 있다.

# 데이터 컬럼의 전체적인 분포 살펴보기

In [8]:
# 카테고리별로 데이터를 나눠 줌
D_columns = [col for col in train_df.columns if 'D_' in col]
S_columns = [col for col in train_df.columns if 'S_' in col]
P_columns = [col for col in train_df.columns if 'P_' in col]
B_columns = [col for col in train_df.columns if 'B_' in col]
R_columns = [col for col in train_df.columns if 'R_' in col]

In [9]:
D_columns

In [10]:
S_columns

In [11]:
P_columns

In [12]:
B_columns

In [13]:
R_columns

In [14]:
print('D 카테고리의 수: ', len(D_columns))
print('S 카테고리의 수: ', len(S_columns))
print('P 카테고리의 수: ', len(P_columns))
print('B 카테고리의 수: ', len(B_columns))
print('R 카테고리의 수: ', len(R_columns))

### observation
- 가장 적은 카테고리는 P다
- 가장 많은 카테고리는 D다

# 데이터 각각의 카테고리 살펴보기

### 우선 각 카테고리의 정보는 아래와 같다.

D_* = 연채(결제불능) 변수

S_* = 소비 변수

P_* = 지불 변수

B_* = 잔액 변수

R_* = 위험 변수

## -연채(D) 변수

In [84]:
display(train_df[D_columns].info(verbose=True))

In [16]:
for columns_n in D_columns:
    if len(train_df[f'{columns_n}'].unique()) < 30:
        print(columns_n)
    else:
        pass

In [17]:
# D 컬럼이 너무 많기 때문에 절반정도로 잘라서 두 파트로 통계정보를 확인
D_columns1 = D_columns[:46]
D_columns2 = D_columns[46:]

In [82]:
train_df['D_114'].unique()

In [18]:
train_df[D_columns1].describe().T

In [19]:
train_df[D_columns2].describe().T

In [41]:
train_df[D_columns1].isnull().sum()

In [64]:
train_df[D_columns2].isnull().sum()

In [46]:
train_df['D_76'].unique()

In [ ]:
train_df[D_columns2]

## observation
- 분류형 데이터로 이루어진 컬럼을 살펴보기 위해서 unique value가 10개 미만인 D 카테고리의 컬럼들만 확인
- 총 10개의 컬럼(D_63, D_64, D_66, D_68, D_87, D_114, D_116, D_117, D_120, D_126)들이 분류형 데이터(실수로 이루어진 분류형 데이터도 존재함)
- 전반적인 데이터에 걸쳐 결측치가 있는 것을 확인할 수 있고 이 데이터들을 처리할 방향을 생각해 봐야할 것 같다.
- 분류형 데이터의 결측치를 처리하는 방향에 대해 생각해 봐야할 것 같다.

## - 소비(S) 변수

In [85]:
display(train_df[S_columns].info(verbose=True))

In [31]:
len(train_df['S_2'].unique()) # S_2 는 날짜 데이터 

In [39]:
np.unique(train_df['S_2'])

In [22]:
for columns_n in S_columns:
    if len(train_df[f'{columns_n}'].unique()) < 30:
        print(columns_n)
    else:
        pass

In [63]:
train_df[S_columns].isnull().sum()

In [24]:
train_df[S_columns].describe().T

## observation
- 소비(S) 변수에는 분류형 데이터가 없음(S_2 날짜 데이터도 연속형 데이터에 포함됨)
- S_2는 날짜데이터로 2017년 3월1일 부터 시작되어 모든 날짜가 포함되어 있다.
- S_3, 7, 9, 22, 24, 25, 26, 27 컬럼에 결측치가 존재한다.

## - 지불(P) 변수

In [51]:
display(train_df[P_columns].info(verbose=True))

In [61]:
train_df['P_2'].unique()

In [65]:
train_df['P_3'].unique()

In [66]:
train_df['P_4'].unique()

In [52]:
for columns_n in P_columns:
    if len(train_df[f'{columns_n}'].unique()) < 30:
        print(columns_n)
    else:
        pass
# 분류형 데이터가 존재하지 않는다

In [54]:
train_df[P_columns].describe().T

## observation
- 지불(P)변수는 모두 연속형 데이터에 포함된다.
- 모두 실수로 이루어진 데이터다.

## - 잔액(B) 변수

In [86]:
display(train_df[B_columns].info(verbose=True))

In [68]:
for columns_n in B_columns:
    if len(train_df[f'{columns_n}'].unique()) < 30:
        print(columns_n)
    else:
        pass

In [69]:
train_df['B_30'].unique()

In [57]:
train_df['B_31'].unique()

In [70]:
train_df['B_38'].unique()

In [71]:
train_df[B_columns].isnull().sum()

## observation
- B_30,31,38 컬럼의 데이터는 분류형 데이터에 포함된다.
- 몇몇 데이터 컬럼에 결측치가 존재한다. 결측치 처리에 대해 생각해볼것

## - 위험(R) 변수

In [88]:
display(train_df[R_columns].info(verbose=True))

In [89]:
for columns_n in R_columns:
    if len(train_df[f'{columns_n}'].unique()) < 30:
        print(columns_n)
    else:
        pass
# 분류형 데이터가 존재하지 않는다.

In [75]:
train_df[R_columns].isnull().sum()

## observation
- 결측치는 R_9, R_20, R_26, R_27 컬럼에 존재한다.
- 모든 데이터가 연속형 실수로 이루어짐.